# Reading, manipulating, and writing data with Astropy

Reading and manipulating tabular data is something every astronomer has to do. Astropy supports reading and writing tables to both ASCII and binary file formats through its [unified input/output interface](http://docs.astropy.org/en/latest/io/unified.html). The core object for representing tables is the `Table`. In this tutorial, we'll learn about the class in general, then do some specific examples of reading and writing both ascii and FITS tables.

For more information about the features presented below, see the
[astropy.table](http://docs.astropy.org/en/stable/table/index.html) documentation.

In [ ]:
# Imports we'll need throughout the tutorial:
import astropy.units as u
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from astropy.table import Table, QTable
from astropy.io import ascii, fits

# Astropy Tables

The astropy [Table](http://docs.astropy.org/en/stable/table/index.html) class provides an extension of NumPy structured arrays for storing and manipulating heterogeneous tables of data. [Table](http://docs.astropy.org/en/stable/table/index.html) supports many convenient features that may be familiar to users of other packages that support table data (e.g., `Pandas`). To highlight a few:

- `Table` objects can be modified in place by adding or removing columns, changing column names, or adding new rows of data.
- `Table`s support dealing with missing values.
- `Table`s support database operations like joins, concatenation, and grouping.
- `Table`s support Astropy [Units and Quantities](http://astropy.readthedocs.org/en/stable/units/index.html), especially with the `QTable` class.

## Creating an astropy `Table` from scratch


Table objects can be constructed in a variety of ways:

- By reading an existing table from a file or web URL
- By adding columns of data one by one
- By adding rows of data one by one
- From an existing data structure in memory, i.e. from a:

  - List of data columns
  - Dict of data columns
  - List of row dicts
  - Numpy homgeneous array or structured array
  - List of row records
  
See the documentation section on [Constructing a table](http://astropy.readthedocs.org/en/stable/table/construct_table.html) for the gory details and plenty of examples.

Let's create our first table object by creating an empty table and start by adding a single column:

In [ ]:
tbl = Table()
tbl['name'] = ['Graham Chapman', 'John Cleese', 'Terry Gilliam', 
               'Eric Idle', 'Terry Jones', 'Michael Palin']

In this case, the column name is given as a string index to the table object, and the data type and number of rows are inferred directly from the data. Once the table has a column, however, any new columns we add must have the same number of rows:

In [ ]:
tbl['age'] = [np.nan, 78, 77, 74, 75, 74]

# whereas, this would fail because it isn't the same length as the table:
# tbl['age'] = [77, 77]

#### Displaying or viewing tables

An instantiated table object can be viewed in a number of ways. In a Jupyter notebook, showing the table will produce a nice HTML representation of the table:

In [ ]:
tbl

If you did the same in a terminal session you get a different view that isn't as pretty but does give a bit more information about the table. In a terminal session, it would look like this:

In [ ]:
print(tbl)

We can access the table column names and data types using the `colnames` and `dtype` attributes:

In [ ]:
tbl.colnames

In [ ]:
tbl.dtype

For long tables, there is also a handy ``show_in_notebook()`` method that allows more interactive exploration like searching, and paged views of the data:

In [ ]:
tbl.show_in_notebook()

#### Accessing sections of the data

We can access the columns and rows the same as we would with numpy structured arrays: string indices access column names, and numeric indices access rows. In the examples below, notice that the returned objects are `Column`, `Row` or `Table` objects depending on the selection:

In [ ]:
tbl['name']  # access entire column

In [ ]:
tbl['name'][1] # row index 1 of column

In [ ]:
tbl[1]  # row object for row index 1

In [ ]:
tbl[1]['name']  # name column of row index 1

In [ ]:
tbl[1:3]  # select a range of rows and return a new table

In [ ]:
tbl[[1, 3]] # select specific row indices and return a new table

In [ ]:
tbl['name', 'age'] # access a subset of columns

#### Masking rows from a table

Like Numpy arrays, we can also use boolean arrays or masks to filter tables. For example, to get the subset of rows where `'age'` is < 75 and finite:

In [ ]:
mask = (tbl['age'] < 75) & np.isfinite(tbl['age'])
tbl[mask]  # Create a new table with just the rows where the mask is True

#### Modifying a table

Once the table exists with defined columns there are a number of ways to modify the table in place.  These are fully documented in the section [Modifying a Table](http://astropy.readthedocs.org/en/stable/table/modify_table.html#modifying-a-table).

We already saw an example above of how to add new columns, but let's add another column. We can also add new rows using the [add_row()](http://astropy.readthedocs.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.add_row) method:

In [ ]:
tbl['british'] = [True, True, False, True, True, True]

In [ ]:
tbl.add_row(('Eddie Izzard', 55, True))  # add a new row to the end
tbl

Notice that the `age` column really has too many output digits given that the age should be an integer.  We can change the display behavior by setting the format using a Python format string:

In [ ]:
tbl['age'].format = '%.0f'
tbl

#### Converting a table to a numpy structured array

Sometimes you may not want or be able to use a `Table` object and prefer to work with a plain numpy array.  This is easily done by passing the table to the `np.array()` constructor.  

Note that this makes a copy of the data.  If you have a huge table and don't want to waste memory, supply `copy=False` to the constructor, but be warned that changing the output numpy array will change the original table.

In [ ]:
np.array(tbl)

Batches of or individual columns and rows can also be converted to plain numpy arrays.

To retrieve column data as a numpy array (as a view):

In [ ]:
tbl['age'].data

Or, to create a copy of the data

In [ ]:
np.array(tbl['age'])

---

# Reading tabular data into `Table` objects

Tabular data in any format supported by the [unified input/ouput interface](http://docs.astropy.org/en/latest/io/unified.html) can be read in as a `Table` object using the [Table.read()](http://astropy.readthedocs.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.read) method. Some notable examples are FITS files with a table, CSV or other delimited ASCII table, or Latex tables. Let's see some examples using files included with this repository:

In [ ]:
!ls -l example_table_data*

In [ ]:
tbl1 = Table.read('data/example_table_data.fits')
tbl1

The `Table.read()` method will try to figure out what type of file and data structure the input file contains:

In [ ]:
!head -n 8 data/example_table_data.csv

In [ ]:
tbl2 = Table.read('data/example_table_data.csv')

but sometimes you have to specify other options to help it out:

In [ ]:
!head -n 8 data/example_table_data.tex

In [ ]:
tbl3 = Table.read('data/example_table_data.tex', format='ascii.aastex')

In [ ]:
!head -n 8 data/example_table_data.txt

In [ ]:
tbl4 = Table.read('data/example_table_data.txt', format='ascii', delimiter='|')

A number of common astronomical ascii formats are supported such as IPAC, sextractor, daophot, and CDS - see [the documentation](http://docs.astropy.org/en/latest/io/unified.html) for a full list, or the docstring:

In [ ]:
Table.read?

--- 

# Writing `Table` objects

To write a table object to disk as a binary or ascii file, use the [Table.write()](http://astropy.readthedocs.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.write) method. This acts just like the `.read()` method, in that in some cases it can automatically guess the format from the filename, but may require specifying the format or other parameters (e.g., delimiter).

An example writing a FITS file:

In [ ]:
tbl1.write('test.fits', overwrite=True)

And with ASCII files of various formats:

In [ ]:
# VO table
tbl1.write('test.vot', format='votable', overwrite=True)
!head -n 30 test.vot

In [ ]:
tbl1.write('test.dat', format='ascii')

In [ ]:
tbl1.write('test.csv', format='ascii.csv')

In [ ]:
tbl1.write('test.ipac', format='ascii')

While the above is often sufficient, it's important to know that the base-line format does *not* support quite a few of the "advanced" features, like inclusion of units.  To do that you'll likely want to use the astropy table "enhanced" CSV:

In [ ]:
tbl1.write('test.ecsv', format='ascii.ecsv')

## Example: retrieving spectroscopic objects from the SDSS

In the example below, we'll query the [SDSS](http://sdss.org) spectroscopic catalog to retrieve object information for all sources within 0.5 degrees of the star HD 69497. We'll then save the table as a FITS file.

In [ ]:
from astroquery.sdss import SDSS
import astropy.coordinates as coord

In [ ]:
sc = coord.SkyCoord.from_name('HD 69497')
sc

In [ ]:
sdss_data = SDSS.query_region(sc, radius=0.5*u.deg, spectro=True)

In [ ]:
sdss_data

In [ ]:
sdss_data.write('SDSS_star.fits', overwrite=True)

---

# Table columns with units

See the [`Quantity` and Tables documentation](http://docs.astropy.org/en/latest/table/mixin_columns.html#quantity-and-qtable) for more information about this section.

Columns of normal `Table` objects can keep track of units, but do not support Quantity-like propagation of units. For example:

In [ ]:
tbl = Table()
tbl['name'] = ['Graham Chapman', 'John Cleese', 'Terry Gilliam', 
               'Eric Idle', 'Terry Jones', 'Michael Palin']
tbl['age'] = [np.nan, 78, 77, 74, 75, 74] * u.year
tbl

Notice that now the 'age' column has a unit `'yr'` listed above its data type.

While `Column` objects can store a unit, they do not act like `Quantity` objects. For example, operations with `Columns` do not propagate or update units like you'd expect:

In [ ]:
(tbl['age'] ** 2).unit

To use `Table`'s that are fully unit aware and use `Quantity` objects to store column data, instead use the `QTable` class. This class behaves similar to the `Table` and can be created in the same way:

In [ ]:
qtbl = QTable()
qtbl['name'] = ['Graham Chapman', 'John Cleese', 'Terry Gilliam', 
                'Eric Idle', 'Terry Jones', 'Michael Palin']
qtbl['age'] = [np.nan, 78, 77, 74, 75, 74] * u.year
qtbl

In [ ]:
type(qtbl['age'])

In [ ]:
(qtbl['age'] ** 2).unit

`Table` and `QTable` objects can be easily converted to the other:

In [ ]:
QTable(tbl)

In [ ]:
Table(qtbl)

---

# Combining tables

Tables can be combined in a number of ways. If two tables have the same columns but different rows, we may want to "stack" them vertically to create a new table that contains all rows. If two tables have different columns, but are row-matched, we may want to "stack" them horizontally to create a new table that contains all columns. We may also want to do something more complicated, like a database join operation. 

All of these are supported between `Table` objects with the `vstack`, `hstack`, and `join` functions. Let's look at a few examples. 

In [ ]:
from astropy.table import vstack, hstack, join

### Combining tables that have the same columns (`vstack`)

Let's first create two tables that share columns but have different rows:

In [ ]:
tbl_a = Table()
tbl_a['cheese'] = ["Red Leicester", "Tilsit", "Caerphilly", "Bel Paese", "Red Windsor"]
tbl_a['in stock'] = ['no', 'no', 'no', 'no', 'no']

tbl_b = Table()
tbl_b['cheese'] = ["Stilton", "Gruyère", "Emmental", "Norwegian Jarlsberg", "Liptauer"]
tbl_b['in stock'] = ['no', 'no', 'no', 'no', 'no']

In [ ]:
tbl_a.colnames, tbl_b.colnames

In [ ]:
cheese = vstack((tbl_a, tbl_b))
cheese

### Combining tables that have the same rows (`hstack`)

Let's first create two tables that are row-matched but have different columns:

In [ ]:
tbl_a = Table()
tbl_a['name'] = ['Arthur',
                 'Lancelot',
                 'Galahad',
                 'Robin']

tbl_b = Table()
tbl_b['title'] = ['King of the Britons',
                  'the Brave',
                  'the Pure',
                  'Not-Quite-So-Brave-as-Sir-Lancelot']

In [ ]:
len(tbl_a), len(tbl_b)

In [ ]:
knights = hstack((tbl_a, tbl_b))
knights

### Database joins (`join`)

Let's first create two tables that share a column but have disparate rows and data. For example, imagine we had one table with source names and metadata (e.g., sky position), and a second table with time series photometry. We might want a new table with all of the photometry, but with the metadata from the first table joined in to each row of the second. 

In [ ]:
from astropy.time import Time

In [ ]:
tbl_a = Table()
tbl_a['name'] = ['Arcturus', 'Rigel', 'Betelgeuse']
tbl_a['ra'] = [213.913315, 78.6344670, 88.7929389] * u.deg
tbl_a['dec'] = [19.1782489, -8.2016383, 7.4070639] * u.deg

tbl_b = Table()
tbl_b['name'] = np.array([], dtype='S10')
tbl_b['time'] = [] * u.hour
tbl_b['mag'] = [] 

tbl_b.add_row(['Arcturus', 15.2523*u.hour, -0.043])
tbl_b.add_row(['Arcturus', 16.955*u.hour, -0.041])
tbl_b.add_row(['Arcturus', 18.011*u.hour, -0.044])
tbl_b.add_row(['Rigel', 20.011*u.hour, 0.12])
tbl_b.add_row(['Betelgeuse', 22.334*u.hour, 0.421])
tbl_b.add_row(['Betelgeuse', 23.930*u.hour, 0.424])

In [ ]:
bright_stars = join(tbl_a, tbl_b, keys='name')
bright_stars

---

# Exercises:

### Read the provided data

We have provided two FITS files in the `data/` directory that contain subsets of files from the Sloan Digital Sky Survey (SDSS) APOGEE survey. APOGEE is a stellar spectroscopic survey that has measured radial velocities and chemical abundances for a few hundred thousand stars. Each star observed was "visited" multiple times, but the visit spectra were combined to obtain a higher signal to noise spectrum for each star. We have provided catalog data measured both from the combined spectrum (`data/apogee-allStar.fits`) and from each individual visit spectrum (`data/apogee-allVisit.fits`) for 100 APOGEE targets. The targets observed by APOGEE were visited a different number of times, so some stars will have as few as 2 or 3 visits, whereas others may have many more.

#### Start by reading in these two files, which each contain a single FITS table:

In [ ]:
# stars = ...
# visits = ...

stars = Table.read('data/apogee-allStar.fits')
visits = Table.read('data/apogee-allVisit.fits')

#### Print out a list of all of the column names for each table:

In [ ]:
print(stars.colnames)
print(visits.colnames)

There are a *lot* of columns in each table! For our purposes, we only care about the following column names:

__`stars`:__
* `APOGEE_ID` - a unique source identifier
* `RA` - the right ascension in degrees
* `DEC` - the declination in degrees
* `NVISITS` - the number of visits for this target

__`visits`:__
* `APOGEE_ID` - the same unique source identifier as above
* `VHELIO` - heliocentric radial velocity
* `JD` - Julian date (time) of the visit observation

---

#### Now explore the data a bit:

- How many rows are in the visits table?
- Display the `stars` table using the notebook-specific method `.show_in_notebook()`
    - Using the search bar from this method, how many stars have the flag `VERY_BRIGHT_NEIGHBOR` set?

In [ ]:
stars.show_in_notebook()

---

#### Modify the stars table:

Let's now add a column to the stars table. The right ascension provided is in decimal degrees. Using `astropy.units`, create a new column called `RA_HOUR` that contains the right ascension in decimal hours instead (_hint: use the unit `u.hourangle` for the angular hour unit_)

#### Delete the `'TARGET_ID'` column from the stars table:

#### Rename the `'RA'` column to `'RA_DEG'`

#### Filter out rows in the `visits` table with bad velocity measurements:

Define a new table `clean_visits` that contains only the rows of the visits table that have good measurements of the velocity (`VHELIO`). A bad velocity will have a value of -9999.

#### Join the `visits` and `stars` tables on the key `APOGEE_ID`